### seq2seq 구조

우리는 seq2seq의 모델 구조를 활용하여 MLE를 수행해 주어진 데이터를 가장 잘 설명하는 파라미터 $\theta$를 찾아내야 합니다.

<br></br>
$$
\hat{\theta} = argmax_\theta P(Y|X;\theta) \\ 
\text{where}\ X = \{x_1, x_2, \dots, x_n \}, Y = \{y_1, y_2, \dots, y_m \}
$$
<br></br>

P(Y|X;\theta)를 최대로 하는 모델 파라미터를 찾는 작업입니다. 이렇게 파라미터에 대한 학습이 완료되면, 사후 확률을 최대로 하는 Y를 찾아야 합니다.

<br></br>
$$
\hat{Y} = argmax_{y \in Y} P(Y|X;\theta)
$$
<br></br>

이를 위해 seq2seq은 크게 3개의 서브 모듈인 인코더, 디코더, 생성자로 구성됩니다.

<br></br>
![](./images/10-2-1-seq2seq.jpg)
<br></br>

#### Encoder

인코더는 주어진 소스 문장인 여러개의 벡터를 입력으로 받아 문장을 함축하는 문장 임베딩 벡터로 만들어냅니다. P(z|X)를 모델링하는 것이라고 볼 수 있습니다. 새로운 형태라기보다는, 텍스트 분류에서 쓰인 RNN 분류 모델과 거의 같다고 볼 수 있습니다. P(z|X)를 모델링하고, 주어진 문장을 매니폴드를 따라 차원 축소하여 해당 도메인의 잠재 공간의 어떤 하나의 점에 투영하는 것입니다.

<br></br>
![](./images/10-2-1-emb.jpg)
<br></br>

기존의 텍스트 본류 문제에서는 모든 정보 (특징^{feature})가 필요하지 않습니다. 예를 들어 감성 분석에서는 *"나는"*과 같은 중립적인 단어는 감성 분류하는 데 필요하지 않으므로 해당 정보를 굳이 간직할 필요가 없었습니다. 따라서 벡터로 만들어내는 과정에서 텍스트 분류를 위해서는 많은 정보를 간직할 필요가 없지만, 기계분석을 위한 문장 임베딩 벡터를 생성하려면 최대한 많은 정보를 간직해야합니다.

<br></br>
$$
h_t^{src} = RNN_{enc}(emb_{src}(x_t),h_{t-1}^{src}) \\
H^{src} = [h_1^{src}; h_2^{src}; \dots; h_n^{src}]
$$
<br></br>

인코더를 수식으로 나타내면 이와 같습니다. [;]는 concatenate작업을 의미합니다. 앞의 수식은 time-step별로 RNN을 통과시켰을을 나타낸 것으로, 실제 구현을 한다면 다음과 같이 전체 time-step을 병렬로 한번에 처리합니다.

<br></br>
$$
H^{src} = RNN_{enc}(emb_{src}(X),h_0^{src})
$$
<br></br>

#### Decoder

디코더 역시 새로운 개념은 아닙니다. 신경망 언어 모델 (NNLM)의 연장선으로 **$조건부\ 신경망\ 언어\ 모델^{conditional\ neural\ network\ language\ model}$** (CNNLM)이라고 할 수 있습니다.

seq2seq 모델의 수식을 time-step에 관해 더 풀어쓰면 다음과 같습니다.

<br></br>
$$
P_\theta(Y|X) = \prod_{t=1}^m P_\theta(y_t|X,y_{<t}) \\
log P_\theta(Y|X) = \sum_{t=1}^m log P_\theta(y_t|X,y_{<t})
$$
<br></br>

이 수식을 보면 RNNLM의 수식에서 조건부 확률 변수 부분에 X가 추가된 것을 확인할 수 있습니다. 즉, 인코더의 결과인 문장 임베딩 벡터와 이전 time-step까지 번역하여 생성한 단어들에 기반하여 현재 time-step의 단어를 생성합니다.

<br></br>
$$
h_t^{tgt} = RNN_{dec}(emb_{tgt}(y_{t-1}),h_{t-1}^{tgt}) \\
\text{where}\ h_0^{tgt} = h_n^{src}, y_0 = BOS
$$
<br></br>

이 수식은 디코더를 나타냅니다. 특기할 점은 디코더 또한 신경망 언어 모델에 속하므로 신경망 언어 모델과 같이 디코더 입력의 초깃값으로써 $y_0$에 BOS 토큰을 입력으로 준다는 점입니다.

#### 생성자

Generator는 디코더에서 각 time-step별로 결과 벡터 $h_t^{tgt}$을 받아 `softmax`를 계산하여 각 타겟 언어의 단어별 확률값을 반환하는 단순한 작업을 수행하는 모듈입니다.

즉 생성자의 결과값은 각 단어가 나타난 확률인 이산 확률 분포가 됩니다. 이때 주의할 점은, 문장의 길이가 |Y| = m일때 맨 마지막 반환되는 단어 $y_m$은 EOS 토큰이 된다는 점입니다. 이 마지막 $y_m$은 디코더 계산의 종료를 나타내므로, 이론상으로는 디코더의 입력으로 들어가는 일이 없습니다.

<br></br>
$$
\hat{y_t} = softmax(linear_{hs\to|V_{tgt}|}(h_t^{tgt})) \\
\hat{y_m} = EOS
$$
<br></br>

### seq2seq의 활용 분야

|활용 분야|입력과 출력|
|-------|---------|
|기계번역|특정 언어 문장을 입력으로 받아 다른 언어의 문장으로 출력|
|챗봇|사용자의 문장 입력을 받아 대답을 출력|
|요약|긴 문장을 입력으로 받아 같은 언어의 요약된 문장으로 출력|
|음성 인식|사용자의 음성을 입력으로 받아 해당 언어의 문자열로 출력|
|독순술|입술 움직임의 동영상을 입력으로 받아 해당 언어의 문장으로 출력|
|이미지 캡셔닝|변형된 seq2seq를 사용하여 이미지 입력으로 받아 그림 설명하는 문장을 출력|

### 파이토치 예제 코드

#### 인코더 클래스

인코더의 RNN은 그 역할이 그렇듯 텍스트 분류기의 코드와 매우 유사합니다. 마찬가지로 Bidirectional LSTM을 사용합니다. 또한 BiDirectional LSTM의 은닉 상태는 기존 단방향 LSTM의 은닉 상태보다 2배로 늘어나므로, hidden_size/2를 사용하였습니다.

In [1]:
import torch
import torch.nn as nn

class Encoder(nn.Module):
    
    def __init__(self, word_vec_dim, hidden_size, n_layers = 4, dropout_p = 0.2):
        super(Encoder, self).__init__()
        
        ## hidden_size is half of original hidden_size, because it is bidirectional
        self.rnn = nn.LSTM(word_vec_dim,
                           int(hidden_size / 2),
                           num_layers = n_layers,
                           dropout = dropout_p,
                           bidirectional = True,
                           batch_first = True)
        
        def forward(self, emb):
            # |emb| = (batch_size, length, word_vec_dim)
            
            if isinstance(emb, tuple):
                x, lengths = emb
                x = pack(x, lengths.tolist(), batch_first = True)
                
            else:
                x = emb
                
            ## |y| = (batch_size, length, hidden_size)
            ## |h[0]| = (num_layers * 2, batch_size, hidden_size / 2)
            y, h = self.rnn(x)
            
            if isinstance(emb, tuple):
                y, _ = unpack(y, batch_first = True)
                
            return y, h

#### pack_padded_sequence 함수

다음은 pack_padded_sequence 함수가 동작하는 모습입니다. 이 함수는 기존의 샘플별 미니 배치를 time-step별로 표현해 주었습니다. PackedSequence로 표현되는 time-step별 미니배치는 각 time-step별 샘플의 숫자를 추가적인 정보로 가집니다. 따라서 이를 위해서는 미니배치내에 가장 긴 길이의 문장부터 차례대로 정렬되어 있어야 합니다.

In [2]:
a = [torch.tensor([1,2,3]), torch.tensor([3,4])]
b = torch.nn.utils.rnn.pad_sequence(a, batch_first = True)
b

tensor([[1, 2, 3],
        [3, 4, 0]])

In [3]:
torch.nn.utils.rnn.pack_padded_sequence(b, batch_first = True, lengths = [3,2])

PackedSequence(data=tensor([1, 3, 2, 4, 3]), batch_sizes=tensor([2, 2, 1]), sorted_indices=None, unsorted_indices=None)

#### 생성자 클래스

In [4]:
class Generator(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(Generator, self).__init__()
        
        self.output = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim = -1)
        
    def forward(self, x):
        ## |x| = (batch_size, length, hidden_size)
        
        ## |y| = (batch_size, length, output_size)
        y = self.softmax(self.output(X))
        
        return y

#### 손실 함수

seq2seq은 기본적으로 각 time-step별로 가장 확률이 높은 단어를 선택하는 분류 문제이므로, **교차 엔트로피**를 손실 함수로 사용합니다. 또한 기본적으로 조건부 언어 모델이라고 볼 수 있으므로 PPL을 통해 번역 모델의 성능을 나타낼수 있습니다.

실제로 구현할때는 "softmax + 교차 엔트로피"를 사용하기보다는 "logsoftmax + 음의 로그 가능도 NLL"를 사용하기도 합니다.

<br></br>
```python
## Default weight for loss equals to 1, but we don't need to get loss for PAD token
## Thus, set a weight for PAD to zero
loss_weight = torch.ones(output_size)
loss_weight[data_loader.PAD] = 0.

## Instead of using Cross-entropy loss, we can use Negative Log-likelihodd (NLL) with log-probability
crit = nn.NLLLoss(weight = loss_weight,
                  reduction = "sum")
```
<br></br>

In [5]:
def get_loss(self, y_hat, y, crit = None):
    ## |y_hat| = (batch_size, length, output_size)
    ## |y| = (batch_size, length)
    
    crit = self.crit if crit is None else crit
    loss = crit(y_hat.contiguous().view(-1, y_hat, size(-1)),
               y.continguous().view(-1))
    
    return loss